# Lehrstellennachweis LENA

Im Lehrstellennachweis LENA unter https://www.berufsberatung.ch/ finden Schulabgänger alle ausgeschriebenen Lehrstellen. In der Schweiz gibt es zur Zeit schon einmal grundsätzlich mehr Lehrstellen als Lernende. Hinzu kommt , dass das Lehrstellenangebot in verschiedenen Branchen nicht immer der Nachfrage entspricht. Je nach Branche bleiben überproportional viele Stellen unbesetzt.  

Im Herbst schreiben die Firmen auf der Plattform jeweils ihre offenen Lehrstellen für den Lehrbeginn im nächsten Sommer (August 2019) aus. In einem ersten Schritt gilt es ein Programm zu schreiben, welches erlaubt die Zahl der offenen Stellen nach Berufen auszulesen. Im Frühling 2019 wird eine erste Zwischenbilanz gezogen, wie viele Stellen besetzt wurden, respektive noch offen sind.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Jeder Beruf verfügt über eine eigene ID. Um den Zusammenhang zwischen Berufsbezeichnung und ID besser zu verstehen, versuche ich als erstes die IDs mit ihren entsprechenden Berufsbezeichnungen auszulesen. Mich interessieren nur die übergeordneten Berufe. zB der "Bekleidungsgestalter/in EFZ", aber nicht die untergeordneten Gruppen wie "Bekleidungsgestalter/in EFZ Kopfbedeckung" oder "Bekleidungsgestalter/in EFZ Pelzbekleidung". Untergruppen sollen nicht gescrapt werden.

In [4]:
url = 'https://www.berufsberatung.ch/dyn/show/2930?'

def get_id_jobnames(url):
    contents = requests.get(url).text   
    soup = BeautifulSoup(contents, 'html.parser')
    list_jobnames = soup.select('ul.full-list a')
    return {
        itrag.attrs['data-id']: itrag.contents[0]
        for itrag in list_jobnames
        if 'EFZ' in itrag.contents[0] or 'EBA' in itrag.contents[0]   
    }

dict_jobnames = get_id_jobnames(url)
dict_jobnames

{'453': 'Abdichter/in EFZ',
 '452': 'Abdichtungspraktiker/in EBA',
 '55': 'Agrarpraktiker/in EBA',
 '902': 'Anlagen- und Apparatebauer/in EFZ',
 '11320': 'Anlagenführer/in EFZ',
 '413': 'Architekturmodellbauer/in EFZ',
 '8082': 'Assistent/in Gesundheit und Soziales EBA',
 '1978': 'Augenoptiker/in EFZ',
 '946': 'Automatiker/in EFZ',
 '982': 'Automatikmonteur/in EFZ',
 '1135': 'Automobil-Assistent/in EBA',
 '1130': 'Automobil-Fachmann/-frau EFZ',
 '1123': 'Automobil-Mechatroniker/in EFZ',
 '7673': 'Bäcker/in-Konditor/in-Confiseur/in EBA',
 '217': 'Bäcker/in-Konditor/in-Confiseur/in EFZ',
 '1074': 'Baumaschinenmechaniker/in EFZ',
 '7659': 'Baupraktiker/in EBA',
 '442': 'Bauwerktrenner/in EFZ',
 '323': 'Bekleidungsgestalter/in EFZ',
 '9663': 'Bekleidungsnäher/in EBA',
 '5': 'Betonwerker/in EFZ',
 '592': 'Boden-Parkettleger/in EFZ',
 '11278': 'Bootbauer/in EFZ',
 '11280': 'Bootfachwart/in EFZ',
 '1298': 'Buchhändler/in EFZ',
 '1149': 'Büchsenmacher/in EFZ',
 '2459': 'Bühnentänzer/in EFZ',
 

Jeder Beruf hat eine eigene url. Wird in der url die ID geändert, gelangt man zu den Suchresultate des entsprechenden Berufs. Die ID in der url wird durch die Variable jobname_id ersetzt.

In [3]:
anzahl_eintraege_url_schema = 'https://www.berufsberatung.ch/dyn/show/2930?lang=de&Idx=0&OrderBy=1&Order=0&PostBackOrder=0&postBack=true&CountResult=122&Total_Idx=111&CounterSearch=1&UrlAjaxWebSearch=%2FLenaWeb%2FAjaxWebSearch&sw_={jobname_id}&LocName=&LocId=&Area=10'

def anzahl_eintraege(jobname_id, url_schema):
    contents = requests.get(url_schema.format(jobname_id=jobname_id)).text
    soup = BeautifulSoup(contents, 'html.parser')
    return int(soup.select('#count-results')[0].contents[0].strip().replace("'", ""))

anzahl_jobs = {
    jobname: anzahl_eintraege(job_id, anzahl_eintraege_url_schema)
    for job_id, jobname in dict_jobnames.items()
}   

anzahl_jobs

{'Abdichter/in EFZ': 135,
 'Abdichtungspraktiker/in EBA': 44,
 'Agrarpraktiker/in EBA': 57,
 'Anlagen- und Apparatebauer/in EFZ': 209,
 'Anlagenführer/in EFZ': 137,
 'Architekturmodellbauer/in EFZ': 0,
 'Assistent/in Gesundheit und Soziales EBA': 478,
 'Augenoptiker/in EFZ': 122,
 'Automatiker/in EFZ': 365,
 'Automatikmonteur/in EFZ': 103,
 'Automobil-Assistent/in EBA': 89,
 'Automobil-Fachmann/-frau EFZ': 607,
 'Automobil-Mechatroniker/in EFZ': 558,
 'Bäcker/in-Konditor/in-Confiseur/in EBA': 74,
 'Bäcker/in-Konditor/in-Confiseur/in EFZ': 338,
 'Baumaschinenmechaniker/in EFZ': 70,
 'Baupraktiker/in EBA': 96,
 'Bauwerktrenner/in EFZ': 20,
 'Bekleidungsgestalter/in EFZ': 134,
 'Bekleidungsnäher/in EBA': 10,
 'Betonwerker/in EFZ': 18,
 'Boden-Parkettleger/in EFZ': 136,
 'Bootbauer/in EFZ': 6,
 'Bootfachwart/in EFZ': 16,
 'Buchhändler/in EFZ': 40,
 'Büchsenmacher/in EFZ': 1,
 'Bühnentänzer/in EFZ': 0,
 'Büroassistent/in EBA': 115,
 'Carrosserielackierer/in EFZ': 170,
 'Carrosseriespengler/

Das Resultat abspeichern als csv-Datei.

In [ ]:
import csv

with open('anzahl_jobs.csv', 'w') as csv_file:
    csv_writer = csv.DictWriter(csv_file, anzahl_jobs.keys())
    csv_writer.writeheader()
    csv_writer.writerow(anzahl_jobs)